In [1]:
import json, codecs, os
import sqlite3

In [2]:
nodes_fpath = os.getcwd() + '/raw_graph_schema/node_properties.json'
rels_fpath = os.getcwd() + '/raw_graph_schema/rel_properties.json'

In [3]:
from typing import List, Dict, Tuple
db_fpath =  '/home/22842219/Desktop/projects_data/spider/database'
sorted_schema = {}
# {"department_management": {'db_id': str, 'tag_names': List(str), 'property_names': [(-1, "*"), (tag_idx, column_name0)...]
# tag_name:{propery_name: []}}}
for fpath in [nodes_fpath, rels_fpath]:
    with codecs.open(fpath, 'r', 'utf-8-sig') as f:
        data = json.load(f)
        for every in data:
            if 'nodeType' in every:
                tag = every['nodeType'][1:]
            elif 'relType' in every:
                tag = every['relType'][1:]
            tag_seps = tag.strip("`").split('.')
            db_id = tag_seps[0]
            
            if db_id not in sorted_schema:
                sorted_schema[db_id] = {'db_id': db_id, 'tag_names_original': [], 'tag_names': [], \
                    'property_names': [(-1, "*")], 'property_types': []}
            tag_name = tag_seps[1]
            if tag_name not in sorted_schema[db_id]:
                sorted_schema[db_id][tag_name]={}  
            if tag_name not in sorted_schema[db_id]['tag_names_original']:
                sorted_schema[db_id]['tag_names_original'].append(tag_name)
                sorted_schema[db_id]['tag_names'].append(tag)

            if bool(every['propertyTypes']):
                if isinstance(every['propertyTypes'], list):
                    sorted_schema[db_id]['property_types'].append('_'.join(every['propertyTypes']))

            # use sqlite cursor to get field values. 
            # we use the concept of picklist to save field values.
        # if db_id == 'department_management':
            db_path = os.path.join(db_fpath, db_id, '{}.sqlite'.format(db_id))
            tag_id = sorted_schema[db_id]['tag_names_original'].index(tag_name)
            if bool(every['propertyName']) and every['propertyName'].lower()!='none':
                sorted_schema[db_id]['property_names'].append(tuple((tag_id, every['propertyName'])))
                fetch_sql = "SELECT DISTINCT `{}` FROM `{}`".format(every['propertyName'], tag_name)
                print(db_id, tag_name, fetch_sql)
                try:
                    conn = sqlite3.connect(db_path)
                    conn.text_factory = bytes
                    c = conn.cursor()
                    c.execute(fetch_sql)
                    picklist = set()
                    # print(c.fetchall())
                    for x in c.fetchall():
                        if bool(x[0]):
                            if isinstance(x[0], str):
                                picklist.add(x[0])
                            elif isinstance(x[0], bytes):
                                try:
                                    picklist.add(x[0].decode("utf-8"))
                                except UnicodeDecodeError:
                                    picklist.add(x[0].decode("latin-1"))
                            else:
                                picklist.add(x[0])
                    picklist = list(picklist)
                finally:
                    conn.close()
                
                sorted_schema[db_id][tag_name][every['propertyName']]=picklist
                sorted_schema[db_id]=sorted_schema[db_id]
            
                


flight_company airport SELECT DISTINCT `IATA` FROM `airport`
flight_company airport SELECT DISTINCT `name` FROM `airport`
flight_company airport SELECT DISTINCT `Country` FROM `airport`
flight_company airport SELECT DISTINCT `ICAO` FROM `airport`
flight_company airport SELECT DISTINCT `id` FROM `airport`
flight_company airport SELECT DISTINCT `City` FROM `airport`
flight_company operate_company SELECT DISTINCT `name` FROM `operate_company`
flight_company operate_company SELECT DISTINCT `id` FROM `operate_company`
flight_company operate_company SELECT DISTINCT `Type` FROM `operate_company`
flight_company operate_company SELECT DISTINCT `Principal_activities` FROM `operate_company`
flight_company operate_company SELECT DISTINCT `Incorporated_in` FROM `operate_company`
flight_company operate_company SELECT DISTINCT `Group_Equity_Shareholding` FROM `operate_company`
flight_company flight SELECT DISTINCT `id` FROM `flight`
flight_company flight SELECT DISTINCT `Pilot` FROM `flight`
flight_c

In [4]:

len(sorted_schema)


155

In [ ]:
sorted_schema

In [5]:
import os
outfpath= os.getcwd() +  '/schema.json'
with codecs.open(outfpath, 'w', encoding='utf-8') as out:
    json.dump(sorted_schema, out, indent=4)

In [ ]:
sp_data_folder =  '/home/22842219/Desktop/phd/SemanticParser4Graph/sp_data_folder'
out='/home/22842219/Desktop/phd/SemanticParser4Graph/semantic_parser/data/text2cypher'
for split in ['train_correct_.json', 'dev_correct_.json']:
    fpth = os.path.join(sp_data_folder,  split )
    print(fpth)
    data = []
    with open(fpth, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
        for every in data:
            if every['answers'] and every['answers']!=[[0]]:
                item = {}
                item['db_id'] = every['db_id']
                item['question'] = every['question']
                item['query'] = every['query']
                item['answers'] = every['answers']
                item['schema']= sorted_schema[every['db_id']]
                data.append(item)


        with open(os.path.join(out, split), 'w') as out:
            json.dump(data, out, indent=4)

                    
